# Project 4 
## Power Outages Data Analysis
### Ethan Lam, A17799139

In [20]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
import time
import folium

### Read in the dataset and clean it

In [2]:
data = pd.read_csv('data/outage.csv')

# Remove first column which is all NaN values
data = data.drop(columns='Unnamed: 0')
data = data.set_index('OBS')
data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",5:00:00 PM,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",6:38:00 PM,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",8:00:00 PM,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,"Tuesday, June 19, 2012",4:30:00 AM,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,"Saturday, July 18, 2015",2:00:00 AM,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,"Tuesday, December 6, 2011",8:00:00 AM,...,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaN,NaN,...,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,"Saturday, August 29, 2009",10:54:00 PM,...,56.65,26.73,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256


Where and when do major power outages tend to occur?

In [3]:
most_power_outage_state = data.groupby('U.S._STATE')['YEAR'].count().sort_values().index[-1]
most_year_power_outage = data.groupby('YEAR')['MONTH'].count().sort_values().index[-1]

Turn time in columns, OUTAGE.START.DATE and OUTAGE.START.TIME, into type pd.Timestamp and combine the two into OUTAGE.START

In [4]:
data['OUTAGE.START.DATE'] = pd.to_datetime(data['OUTAGE.START.DATE'], format='%A, %B %d, %Y')
data['OUTAGE.START.TIME'] = pd.to_datetime(data['OUTAGE.START.TIME'], format='%I:%M:%S %p').dt.time

In [5]:
data['OUTAGE.START'] = data['OUTAGE.START.DATE'].dt.strftime('%A, %B %d, %Y') + ' ' + data['OUTAGE.START.TIME'].astype(str)
data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,...,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,"Friday, July 01, 2011 17:00:00"
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,...,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,"Sunday, May 11, 2014 18:38:00"
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,...,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,"Tuesday, October 26, 2010 20:00:00"
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,...,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,"Tuesday, June 19, 2012 04:30:00"
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,...,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,"Saturday, July 18, 2015 02:00:00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,08:00:00,...,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,"Tuesday, December 06, 2011 08:00:00"
1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaT,NaT,...,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaN
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,22:54:00,...,26.73,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,"Saturday, August 29, 2009 22:54:00"


Combine OUTAGE.RESTORATION.DATE and OUTAGE.RESTORATION.TIME into a new column, OUTAGE.RESTORATION

In [8]:
data['OUTAGE.RESTORATION.DATE'] = pd.to_datetime(data['OUTAGE.RESTORATION.DATE'], format='%A, %B %d, %Y')
data['OUTAGE.RESTORATION.TIME'] = pd.to_datetime(data['OUTAGE.RESTORATION.TIME'], format='%I:%M:%S %p').dt.time
data['OUTAGE.RESTORATION'] = data['OUTAGE.RESTORATION.DATE'].dt.strftime('%A, %B %d, %Y') + ' ' + data['OUTAGE.RESTORATION.TIME'].astype(str)
data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,"Friday, July 01, 2011 17:00:00","Sunday, July 03, 2011 20:00:00"
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,"Sunday, May 11, 2014 18:38:00","Sunday, May 11, 2014 18:39:00"
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,"Tuesday, October 26, 2010 20:00:00","Thursday, October 28, 2010 22:00:00"
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,"Tuesday, June 19, 2012 04:30:00","Wednesday, June 20, 2012 23:00:00"
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,"Saturday, July 18, 2015 02:00:00","Sunday, July 19, 2015 07:00:00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,08:00:00,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,"Tuesday, December 06, 2011 08:00:00","Tuesday, December 06, 2011 20:00:00"
1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaT,NaT,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaN,NaN
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,22:54:00,...,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,"Saturday, August 29, 2009 22:54:00","Saturday, August 29, 2009 23:53:00"


In [40]:
relevant = data[['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL', 'RES.SALES', 'OUTAGE.START', 'OUTAGE.RESTORATION', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'TOTAL.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS', 'POPPCT_URBAN', 'POPDEN_URBAN', 'AREAPCT_URBAN']]

In [41]:
pd.set_option('display.max_columns', None)
relevant

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,RES.SALES,OUTAGE.START,OUTAGE.RESTORATION,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,TOTAL.PRICE,TOTAL.SALES,TOTAL.CUSTOMERS,POPPCT_URBAN,POPDEN_URBAN,AREAPCT_URBAN
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,severe weather,NaN,2332915.0,"Friday, July 01, 2011 17:00:00","Sunday, July 03, 2011 20:00:00",3060.0,NaN,70000.0,9.28,6562520.0,2595696,73.27,2279.0,2.14
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,intentional attack,vandalism,1586986.0,"Sunday, May 11, 2014 18:38:00","Sunday, May 11, 2014 18:39:00",1.0,NaN,NaN,9.28,5284231.0,2640737,73.27,2279.0,2.14
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,severe weather,heavy wind,1467293.0,"Tuesday, October 26, 2010 20:00:00","Thursday, October 28, 2010 22:00:00",3000.0,NaN,70000.0,8.15,5222116.0,2586905,73.27,2279.0,2.14
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,severe weather,thunderstorm,1851519.0,"Tuesday, June 19, 2012 04:30:00","Wednesday, June 20, 2012 23:00:00",2550.0,NaN,68200.0,9.19,5787064.0,2606813,73.27,2279.0,2.14
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,severe weather,NaN,2028875.0,"Saturday, July 18, 2015 02:00:00","Sunday, July 19, 2015 07:00:00",1740.0,250.0,250000.0,10.43,5970339.0,2673531,73.27,2279.0,2.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,public appeal,NaN,488853.0,"Tuesday, December 06, 2011 08:00:00","Tuesday, December 06, 2011 20:00:00",720.0,155.0,34500.0,7.56,1313678.0,394394,59.90,2192.2,0.27
1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,fuel supply emergency,Coal,NaN,NaN,NaN,NaN,1650.0,NaN,NaN,NaN,366037,59.90,2192.2,0.27
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,islanding,NaN,337874.0,"Saturday, August 29, 2009 22:54:00","Saturday, August 29, 2009 23:53:00",59.0,84.0,NaN,7.67,924051.0,436229,56.65,2038.3,0.30


In [45]:
month_dict = {1.0: 'January', 2.0: 'February', 3.0: 'March', 4.0: 'April', 5.0: 'May', 6.0: 'June', 7.0: 'July', 8.0: 'August', 9.0: 'September', 10.0: 'October', 11.0: 'November', 12.0: 'December'}
relevant = relevant.copy()
relevant['MONTH'] = relevant['MONTH'].apply(lambda x: month_dict[x] if x in month_dict else x)


Now that our data is cleaned up, let's explore variables that would have
correlation to power outages

In [46]:
num_causes = relevant.groupby('CAUSE.CATEGORY')['YEAR'].count().sort_values()
causes = px.bar(num_causes)
causes.show()

In [47]:
num_states = relevant.groupby('U.S._STATE')['YEAR'].count().sort_values()
states = px.bar(num_states)
states.show()

In [48]:
num_year = relevant.groupby('YEAR')['MONTH'].count()
year = px.line(num_year)
year.show()

In [66]:
month_order = ["January", "February", "March", "April", "May", "June", 
               "July", "August", "September", "October", "November", "December"]
num_month = relevant.groupby('MONTH')['YEAR'].count()
num_month = num_month.sort_index(key=lambda x: x.map(lambda month: month_order.index(month)))
month = px.line(num_month)
month.show()


Now move on to two variable analysis

In [98]:
relevant.loc[relevant['CAUSE.CATEGORY'] == 'public appeal']

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,RES.SALES,OUTAGE.START,OUTAGE.RESTORATION,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,TOTAL.PRICE,TOTAL.SALES,TOTAL.CUSTOMERS,POPPCT_URBAN,POPDEN_URBAN,AREAPCT_URBAN
OBS,,,,,,,,,,,,,,,,,,,,,
30,2012,June,Tennessee,TN,SERC,Central,-0.1,public appeal,NaN,3426874.0,"Saturday, June 30, 2012 15:00:00","Monday, July 02, 2012 12:00:00",2700.0,NaN,NaN,9.55,8395017.0,3189570,66.39,1450.3,7.05
60,2010,August,Wisconsin,WI,MRO,East North Central,-1.2,public appeal,NaN,2315414.0,"Thursday, August 12, 2010 15:42:00","Thursday, August 12, 2010 22:10:00",388.0,30.0,7600.0,10.17,6717006.0,2937600,70.15,2123.3,3.47
77,2011,July,Michigan,MI,RFC,East North Central,-0.3,public appeal,NaN,4008491.0,"Thursday, July 21, 2011 12:32:00","Friday, July 22, 2011 06:30:00",1078.0,8881.0,NaN,11.04,10846247.0,4783420,74.57,2034.1,6.41
174,2014,February,Texas,TX,TRE,South,-0.5,public appeal,NaN,11797374.0,"Thursday, February 06, 2014 15:35:00","Friday, February 07, 2014 11:30:00",1195.0,NaN,NaN,8.85,30082500.0,11672996,84.70,2435.3,3.35
184,2015,August,Texas,TX,TRE,South,1.4,public appeal,NaN,17143154.0,"Thursday, August 13, 2015 15:15:00","Thursday, August 13, 2015 19:00:00",225.0,NaN,NaN,8.90,40357061.0,11849725,84.70,2435.3,3.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,2006,August,Oklahoma,OK,RFC,South,0.3,public appeal,NaN,2892233.0,"Monday, August 07, 2006 13:00:00","Monday, August 07, 2006 18:00:00",300.0,75.0,NaN,8.09,6206348.0,1884744,66.24,1901.7,1.90
1425,2011,August,Oklahoma,OK,SPP,South,-0.6,public appeal,NaN,3179850.0,"Wednesday, August 03, 2011 16:29:00","Wednesday, August 03, 2011 23:40:00",431.0,300.0,NaN,8.52,6686268.0,1950247,66.24,1901.7,1.90
1513,2011,June,Kansas,KS,SPP,South,-0.3,public appeal,NaN,1466261.0,"Sunday, June 26, 2011 16:46:00","Monday, June 27, 2011 07:59:00",913.0,NaN,NaN,9.45,3858223.0,1461241,74.20,2176.5,1.19


In [80]:
duration_vs_sales = px.scatter(relevant, x='TOTAL.SALES', y='OUTAGE.DURATION')
duration_vs_sales.show()

In [100]:
demandloss_vs_cause = px.scatter(relevant, x='CUSTOMERS.AFFECTED', y='OUTAGE.DURATION')
demandloss_vs_cause